In [473]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [474]:
stats = pd.read_csv('footywire_stats.csv')
games = pd.read_csv('footywire_games.csv')

In [475]:
import re

# Remove non-alphanumeric characters except space, hyphen, and apostrophe
stats['Player'] = stats['Player'].str.replace(r"[^\w\s'-]", '', regex=True).str.strip()

First I will create columns for which team won the match. These will be used as the target feature of my initial model.

In [476]:
games['AwayWin'] = games['Away.Points'] > games['Home.Points']
games['HomeWin'] = games['Away.Points'] < games['Home.Points']
games['Draw'] = games['Away.Points'] == games['Home.Points']
games[['AwayWin', 'HomeWin', 'Draw']] = games[['AwayWin', 'HomeWin', 'Draw']].astype(int)

In [477]:
games['Date'] = pd.to_datetime(games['Date'])
games['Season'] = games['Date'].dt.year

In [478]:
merge_stats = stats[['Match_id', 'Round', 'Team', 'Season']].rename(columns={'Team': 'Home.Team'})
games = games.merge(merge_stats, on=['Round', 'Home.Team', 'Season']).drop_duplicates().reset_index(drop=True)

In [479]:
finals_map = {
            'Qualifying Final': 25,
            'Elimination Final': 25,
            'Semi Final': 26,
            'Preliminary Final': 27,
            'Preliminary Finals': 27,
            'Grand Final': 28}
games['Round'] = games['Round'].replace(finals_map)
games['Round'] = games['Round'].replace(r'Round (\d+)', r'\1', regex=True)
games['Round'] = games['Round'].astype(int)

stats['Round'] = stats['Round'].replace(finals_map)
stats['Round'] = stats['Round'].replace(r'Round (\d+)', r'\1', regex=True)
stats['Round'] = stats['Round'].astype(int)


Objective: 
Build model that can predict the outcomce of a game.
Features:


- **Net Team Rating:** This will be calculated by finding the offensive and defensive ratings of each team.
teamA net rating = teamA offensive rating - teamB defensive rating.

- **Net Team Form:** This will be calculated by finding the difference in winrate for both teams over the last 5 games.
teamA net form = teamA winrate - teamB winrate.

- **Winrate against opposition:** This will be calculated by finding the winrate against the opposition over the last 5 games they played each other.
teamA winrate = teamA won against teamB / teamA lost against teamB

- **Winrate at venue:** This will be calculated by finding the winrate of a team at the current venue over the last 5 games they played there.


- Final - how many finals a team has played in the last 2 years


**Step 1. Build team offensive rating**

The offensive rating of a team will be calculated based on the player offensive ratings in the team. If a key player is missing for example, then the offensive rating will decrease, and vice versa. 

Features used in calculating player offensive ratings:

- Handballs: secondary ball movement
- Marks: possession retention
- Goals: primary scoring
- Behinds: partial scoring
- Hitouts: minor offensive contribution
- Inside 50s: advancing play
- Clearances: initiating play
- Frees For: advantage gained
- Marks Inside 50: scoring opportunity
- Goal Assists: direct scoring contribution
- Score Involvements: broader scoring impact
- Metres Gained: small weight for territory
- Effective Disposals: quality ball use
- Normalize by % Time on Ground

The stats of the player for the last 10 games will be used to calculate this score

In [183]:
stats['Round'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 24,  0])

In [184]:
stats.columns

Index(['Date', 'Season', 'Round', 'Venue', 'Player', 'Team', 'Opposition',
       'Status', 'Match_id', 'GA', 'CP', 'UP', 'ED', 'DE', 'CM', 'MI5',
       'One.Percenters', 'BO', 'TOG', 'K', 'HB', 'D', 'M', 'G', 'B', 'T', 'HO',
       'I50', 'CL', 'CG', 'R50', 'FF', 'FA', 'AF', 'SC', 'CCL', 'SCL', 'SI',
       'MG', 'TO', 'ITC', 'T5'],
      dtype='object')

In [455]:
def get_last_10_rounds(matchid, team, stats):
    team_stats = stats[stats['Team'] == team].copy()
    current = team_stats[team_stats['Match_id'] == matchid]['Round'].max()
    current_season_rounds = []
    remaining_rounds = 0
    previous_season = None
    current_year = team_stats.loc[team_stats['Match_id'] == matchid, 'Season'].values[0]
    years = [current_year]
    for i in range(1, 11):
        if current-i > -1:
            current_season_rounds.append(current-i)
        else:
            previous_season = True
            remaining_rounds += 1
    if previous_season and current_year != 2012:
        previous_season_rounds = []
        previous_year = current_year - 1
        previous_season = team_stats[team_stats['Season'] == previous_year]
        last_round = previous_season['Round'].max()
        for i in range(0, remaining_rounds + 1):
            previous_season_rounds.append(last_round-i)
        years = [current_year, previous_year]
        return [current_season_rounds, previous_season_rounds, years, team_stats]
    else:
        return [current_season_rounds, None, years, team_stats]

In [186]:
get_last_10_rounds(10970, 'Brisbane', stats)

[[27, 26, 25, 24, 23, 22, 21, 20, 19, 18],
 None,
 [np.int64(2023)],
              Date  Season  Round                 Venue           Player  \
 132    2015-04-04    2015      1                 Gabba      Dayne Beams   
 133    2015-04-04    2015      1                 Gabba      Daniel Rich   
 134    2015-04-04    2015      1                 Gabba      Dayne Zorko   
 135    2015-04-04    2015      1                 Gabba      Claye Beams   
 136    2015-04-04    2015      1                 Gabba      Jack Redden   
 ...           ...     ...    ...                   ...              ...   
 98774  2025-07-26    2025     20  People First Stadium  Charlie Cameron   
 98775  2025-07-26    2025     20  People First Stadium       Darcy Fort   
 98776  2025-07-26    2025     20  People First Stadium  Oscar McInerney   
 98777  2025-07-26    2025     20  People First Stadium    Conor McKenna   
 98778  2025-07-26    2025     20  People First Stadium     Logan Morris   
 
            Team 

In [187]:
def find_player_off(matchid, team, stats):
    last_10_rounds = get_last_10_rounds(matchid, team, stats)
    years = last_10_rounds[2]
    current_year_rounds = last_10_rounds[0]
    previous_year_rounds = last_10_rounds[1]
    scores_stats = last_10_rounds[3]
    if previous_year_rounds:
        scores_stats = scores_stats[((scores_stats['Season'] == years[0]
                                     ) & (scores_stats['Round'].isin(current_year_rounds)
                                         ) | (scores_stats[
                                              'Season'] == years[1]
                                             ) & (scores_stats['Round'].isin(previous_year_rounds)))]
    else:
        scores_stats = scores_stats[(scores_stats['Season'] == years[0]
                                     ) & (scores_stats['Round'].isin(current_year_rounds)
                                         )]
    scores_stats = scores_stats.groupby('Player', as_index=False)[['K', 'M', 'HB', 'G', 'B', 'HO', 'I50', 'CL', 'FF', 'MI5', 'GA', 'SI', 'MG', 'ED', 'TOG']].mean()
    scores_stats['OffensiveRating'] = (
        2.0 * scores_stats['K'] +                    # Kicks: key ball movement
        1.0 * scores_stats['HB'] +                   # Handballs: secondary ball movement
        1.5 * scores_stats['M'] +                    # Marks: possession retention
        6.0 * scores_stats['G'] +                    # Goals: primary scoring
        1.0 * scores_stats['B'] +                    # Behinds: partial scoring
        0.5 * scores_stats['HO'] +                   # Hitouts: minor offensive contribution
        3.0 * scores_stats['I50'] +                  # Inside 50s: advancing play
        2.0 * scores_stats['CL'] +                   # Clearances: initiating play
        1.0 * scores_stats['FF'] +                   # Frees For: advantage gained
        4.0 * scores_stats['MI5'] +                  # Marks Inside 50: scoring opportunity
        3.0 * scores_stats['GA'] +                   # Goal Assists: direct scoring contribution
        2.5 * scores_stats['SI'] +                   # Score Involvements: broader scoring impact
        0.01 * scores_stats['MG'] +                  # Metres Gained: small weight for territory
        1.5 * scores_stats['ED']) * (scores_stats['TOG'] / 100.0)                # Normalize by % Time on Ground
    current_players = stats[(stats['Match_id'] == matchid) & (stats['Team'] == team)]['Player']
    scores_stats = scores_stats[scores_stats['Player'].isin(current_players)]
    return scores_stats[['Player', 'OffensiveRating']]

In [188]:
find_player_off(10970, 'Brisbane', stats)

,Player,OffensiveRating
0,Brandon Starcevich,45.391249
1,Callum Ah Chee,47.967566
2,Cameron Rayner,57.760022
3,Charlie Cameron,75.021138
4,Conor McKenna,67.750667
6,Darcy Gardiner,51.093000
7,Darcy Wilmot,66.350604
8,Dayne Zorko,76.653191
9,Deven Robertson,20.090253
10,Eric Hipwood,69.413535


These are the offensive player scores for the 2023 Grand Final. They are calculated using the stat averages of the players for the last 10 games.

**Step 2. Build Team Defensive Rating**

The defensive rating of a team will be calculated based on the player defensive ratings in the team. If a key defensive player is missing, the team's defensive rating will decrease, and vice versa.

Features used in calculating player defensive ratings:

- Tackles: disrupting opposition possession
- One-Percenters: defensive acts like spoils or knock-ons
- Rebound 50s: repelling opposition attacks
- Contested Marks: winning possession under pressure
- Intercepts: regaining possession from opposition disposals
- Contested Possessions: winning ball in contested situations
- Centre Clearances: preventing opposition momentum from centre bounces
- Stoppage Clearances: preventing opposition momentum from stoppages
- Marks: possession retention in defense
- Kicks: effective ball movement out of defense
- Handballs: secondary ball movement out of defense
- Disposal Efficiency: quality of ball use under pressure
- Turnovers (negative): losing possession to opposition
- Clangers (negative): mistakes like turnovers or poor decisions
- Free Kicks Against (negative): penalties conceded
- Normalize by % Time on Ground

The stats of the player for the last 10 games will be used to calculate this score.

In [189]:
def find_player_def(matchid, team, stats):
    last_10_rounds = get_last_10_rounds(matchid, team, stats)
    years = last_10_rounds[2]
    current_year_rounds = last_10_rounds[0]
    previous_year_rounds = last_10_rounds[1]
    scores_stats = last_10_rounds[3]
    if previous_year_rounds:
        scores_stats = scores_stats[((scores_stats['Season'] == years[0]) & 
                                    (scores_stats['Round'].isin(current_year_rounds))) | 
                                   ((scores_stats['Season'] == years[1]) & 
                                    (scores_stats['Round'].isin(previous_year_rounds)))]
    else:
        scores_stats = scores_stats[(scores_stats['Season'] == years[0]) & 
                                   (scores_stats['Round'].isin(current_year_rounds))]
    scores_stats = scores_stats.groupby('Player', as_index=False)[[
        'K', 'HB', 'M', 'CP', 'CM', 'One.Percenters', 'T', 'R50', 
        'CCL', 'SCL', 'CG', 'FA', 'TOG', 'DE', 'ITC', 'TO']].mean()
    scores_stats['DefensiveRating'] = (
        4 * scores_stats['T'] + 
        3 * scores_stats['One.Percenters'] + 
        2.5 * scores_stats['R50'] + 
        2.5 * scores_stats['CM'] + 
        2.5 * scores_stats['ITC'] + 
        1.5 * scores_stats['CP'] + 
        1.5 * scores_stats['CCL'] + 
        1.0 * scores_stats['SCL'] + 
        0.5 * scores_stats['M'] + 
        0.3 * scores_stats['K'] + 
        0.3 * scores_stats['HB'] + 
        0.1 * scores_stats['DE'] - 
        2.5 * scores_stats['TO'] - 
        2 * scores_stats['CG'] - 
        1.5 * scores_stats['FA']
    ) * (scores_stats['TOG'] / 100)
    current_players = stats[(stats['Match_id'] == matchid) & (stats['Team'] == team)]['Player']
    scores_stats = scores_stats[scores_stats['Player'].isin(current_players)]
    return scores_stats[['Player', 'DefensiveRating']]

In [137]:
find_player_def(10970, 'Brisbane', stats)

,Player,DefensiveRating
0,Brandon Starcevich,32.388240
1,Callum Ah Chee,23.431647
2,Cameron Rayner,22.864519
3,Charlie Cameron,27.903585
4,Conor McKenna,27.628030
6,Darcy Gardiner,60.570000
7,Darcy Wilmot,38.573167
8,Dayne Zorko,31.783154
9,Deven Robertson,16.569992
10,Eric Hipwood,17.522730


In [190]:
def find_team_scores(matchid, team, stats):
    # Square the individual ratings before taking the mean
    player_def_scores = find_player_def(matchid, team, stats)['DefensiveRating']
    player_off_scores = find_player_off(matchid, team, stats)['OffensiveRating']
    
    player_def = float(round((player_def_scores ** 2).mean(), 2))
    player_off = float(round((player_off_scores ** 2).mean(), 2))
    
    return [matchid, team, player_off, player_def]

In [191]:
find_team_scores(10970, 'Brisbane', stats)

[10970, 'Brisbane', 4538.71, 1412.68]

In [192]:
from sklearn.preprocessing import MinMaxScaler
def add_team_and_net_ratings(games: pd.DataFrame, stats: pd.DataFrame, years) -> pd.DataFrame:
    games_subset = games[games['Season'].isin(years)].copy()

    # Compute team ratings once per game/team
    games_subset['AwayStats'] = games_subset.apply(
        lambda row: find_team_scores(row['Match_id'], row['Away.Team'], stats),
        axis=1
    )
    games_subset['HomeStats'] = games_subset.apply(
        lambda row: find_team_scores(row['Match_id'], row['Home.Team'], stats),
        axis=1
    )
    # Extract offensive and defensive ratings
    games_subset[['OffensiveRatingAway', 'DefensiveRatingAway']] = games_subset['AwayStats'].apply(lambda x: pd.Series(x[2:4]))
    games_subset[['OffensiveRatingHome', 'DefensiveRatingHome']] = games_subset['HomeStats'].apply(lambda x: pd.Series(x[2:4]))
    # Calculate team ratings
    games_subset['AwayTeamRating'] = games_subset['OffensiveRatingAway'] - games_subset['DefensiveRatingHome']
    games_subset['HomeTeamRating'] = games_subset['OffensiveRatingHome'] - games_subset['DefensiveRatingAway']
    # Calculate net rating (away - home)
    games_subset['AwayNetRating'] = games_subset['AwayTeamRating'] - games_subset['HomeTeamRating']
    games_subset['HomeNetRating'] = games_subset['HomeTeamRating'] - games_subset['AwayTeamRating']
    scaler = MinMaxScaler(feature_range=(0, 1))
    games_subset[['HomeNetRating', 'AwayNetRating']] = scaler.fit_transform(games_subset[['HomeNetRating', 'AwayNetRating']])
    return games_subset.drop(columns=['HomeStats', 'AwayStats'])

In [193]:
add_team_and_net_ratings(games, stats, [2021])

,Date,Time,Round,Venue,Home.Team,Away.Team,Home.Points,Away.Points,AwayWin,HomeWin,...,Season,Match_id,OffensiveRatingAway,DefensiveRatingAway,OffensiveRatingHome,DefensiveRatingHome,AwayTeamRating,HomeTeamRating,AwayNetRating,HomeNetRating
26312,2021-03-18,19:25,1,MCG,Richmond,Carlton,105,80,0,1,...,2021,10327,2960.82,1248.61,2981.66,1020.16,1940.66,1733.05,0.510444,0.489556
26335,2021-03-19,19:50,1,MCG,Collingwood,Western Bulldogs,53,69,1,0,...,2021,10328,4636.47,1428.05,2697.29,1198.29,3438.18,1269.24,0.833952,0.166048
26357,2021-03-20,13:45,1,MCG,Melbourne,Fremantle,80,58,0,1,...,2021,10329,3446.85,1271.36,3278.35,1164.02,2282.83,2006.99,0.521698,0.478302
26379,2021-03-20,16:05,1,Adelaide Oval,Adelaide,Geelong,103,91,0,1,...,2021,10330,3835.71,1014.06,2480.47,1007.49,2828.22,1466.41,0.700821,0.299179
26402,2021-03-20,18:45,1,Gabba,Brisbane,Sydney,94,125,1,0,...,2021,10332,3435.69,1311.86,3298.70,1356.19,2079.50,1986.84,0.491484,0.508516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30856,2021-09-03,17:50,26,Optus Stadium,Geelong,GWS,103,68,0,1,...,2021,10539,3894.09,1544.91,5310.30,1341.91,2552.18,3765.39,0.276090,0.723910
30879,2021-09-04,19:20,26,Gabba,Brisbane,Western Bulldogs,78,79,1,0,...,2021,10540,4660.60,1375.08,5413.68,1796.74,2863.86,4038.60,0.282435,0.717565
30902,2021-09-10,17:50,27,Optus Stadium,Melbourne,Geelong,125,42,0,1,...,2021,10541,5470.11,1297.14,5226.15,1607.42,3862.69,3929.01,0.465261,0.534739
30925,2021-09-11,19:10,27,Adelaide Oval,Port Adelaide,Western Bulldogs,45,116,1,0,...,2021,10542,4711.13,1456.13,5217.46,1495.58,3215.55,3761.33,0.386178,0.613822


In [194]:
def get_last_10_games(matchid, team, games):
    team_games = games[(games['Home.Team'] == team) | (games['Away.Team'] == team)].copy()
    
    current = team_games[team_games['Match_id'] == matchid]['Round'].max()
    current_season_rounds = []
    remaining_rounds = 0
    previous_season = None
    current_year = team_games.loc[team_games['Match_id'] == matchid, 'Season'].values[0]
    years = [current_year]
    
    for i in range(1, 11):
        if current-i > -1:
            current_season_rounds.append(current-i)
        else:
            previous_season = True
            remaining_rounds += 1
    if previous_season and current_year != 2012:
        previous_season_rounds = []
        previous_year = current_year - 1
        previous_season = team_games[team_games['Season'] == previous_year]
        last_round = previous_season['Round'].max()
        for i in range(0, remaining_rounds + 1):
            previous_season_rounds.append(last_round-i)
        years = [current_year, previous_year]     
        return [current_season_rounds, previous_season_rounds, years, team_games]
    else:
        return [current_season_rounds, None, years, team_games]

In [195]:
get_last_10_games(10970, 'Brisbane', games)

[[27, 26, 25, 24, 23, 22, 21, 20, 19, 18],
 None,
 [np.int32(2023)],
             Date   Time  Round                  Venue        Home.Team  \
 66    2015-04-04  18:20      1                  Gabba         Brisbane   
 374   2015-04-12  16:40      2         Marvel Stadium  North Melbourne   
 506   2015-04-18  19:20      3                  Gabba         Brisbane   
 726   2015-04-26  13:10      4                  Gabba         Brisbane   
 858   2015-05-02  16:35      5   People First Stadium       Gold Coast   
 ...          ...    ...    ...                    ...              ...   
 48336 2025-06-20  19:40     15          GMHBA Stadium          Geelong   
 48748 2025-07-05  19:35     17                  Gabba         Brisbane   
 48840 2025-07-10  19:30     18         Marvel Stadium          Carlton   
 49070 2025-07-18  19:40     19                  Gabba         Brisbane   
 49323 2025-07-26  13:20     20   People First Stadium       Gold Coast   
 
               Away.Team  Hom

In [196]:
def form(matchid, team, games):
    last_10_rounds = get_last_10_games(matchid, team, games)
    years = last_10_rounds[2]
    current_year_rounds = last_10_rounds[0]
    previous_year_rounds = last_10_rounds[1]
    games_10 = last_10_rounds[3]
    if previous_year_rounds:
        games_10 = games_10[((games_10['Season'] == years[0]
                                     ) & (games_10['Round'].isin(current_year_rounds)
                                         ) | (games_10['Season'] == years[1]
                                             ) & (games_10['Round'].isin(previous_year_rounds)))]
    else:
        games_10 = games_10[(games_10['Season'] == years[0]
                                     ) & (games_10['Round'].isin(current_year_rounds)
                                         )]
    games_10['TeamWin'] = ((games_10['Away.Team'] == team) & (games_10['AwayWin'] == True)) | ((games_10['Home.Team'] == team) & (games_10['HomeWin'] == True))
    winrate = games_10['TeamWin'].mean()
    return winrate

In [197]:
form(10970, 'Brisbane', games)

C:\Users\maxma\AppData\Local\Temp\ipykernel_42108\992251843.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_10['TeamWin'] = ((games_10['Away.Team'] == team) & (games_10['AwayWin'] == True)) | ((games_10['Home.Team'] == team) & (games_10['HomeWin'] == True))


np.float64(0.7777777777777778)

In [198]:
def add_winrate_columns(games: pd.DataFrame, years) -> pd.DataFrame:
    games_subset = games[games['Season'].isin(years)].copy()


    # Calculate form (win rate in last 10 games)
    games_subset['AwayForms'] = games_subset.apply(
        lambda row: form(row['Match_id'], row['Away.Team'], games),
        axis=1
    )
    
    games_subset['HomeForms'] = games_subset.apply(
        lambda row: form(row['Match_id'], row['Home.Team'], games),
        axis=1
    )

    # Net form (away - home, and vice versa)
    games_subset['AwayNetForm'] = games_subset['AwayForms'] - games_subset['HomeForms']
    games_subset['HomeNetForm'] = games_subset['HomeForms'] - games_subset['AwayForms']

    return games_subset.drop(columns = ['AwayForms', 'HomeForms'])


In [199]:
add_winrate_columns(games, [2023])

,Date,Time,Round,Venue,Home.Team,Away.Team,Home.Points,Away.Points,AwayWin,HomeWin,Draw,Season,Match_id,AwayNetForm,HomeNetForm
35625,2023-03-16,19:20,1,MCG,Richmond,Carlton,58,58,0,0,1,2023,10751,-0.255556,0.255556
35648,2023-03-17,19:40,1,MCG,Geelong,Collingwood,103,125,1,0,0,2023,10752,-0.333333,0.333333
35671,2023-03-18,13:45,1,Marvel Stadium,North Melbourne,West Coast,87,82,0,1,0,2023,10753,-0.011111,0.011111
35694,2023-03-18,16:05,1,Adelaide Oval,Port Adelaide,Brisbane,126,72,0,1,0,2023,10754,0.166667,-0.166667
35717,2023-03-18,19:00,1,People First Stadium,Gold Coast,Sydney,61,110,1,0,0,2023,10756,0.475000,-0.475000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40475,2023-09-15,19:50,26,MCG,Melbourne,Carlton,71,73,1,0,0,2023,10966,0.200000,-0.200000
40497,2023-09-16,19:10,26,Adelaide Oval,Port Adelaide,GWS,70,93,1,0,0,2023,10967,0.300000,-0.300000
40520,2023-09-22,19:50,27,MCG,Collingwood,GWS,58,57,0,1,0,2023,10968,0.133333,-0.133333
40543,2023-09-23,17:15,27,Gabba,Brisbane,Carlton,79,63,0,1,0,2023,10969,0.122222,-0.122222


Next I will engineer a feature that rates how well a team performed against the opposition team the last 5 games they played each other.

In [200]:
def get_last_5_matchups(matchid, team1, team2, games):
    team_games = games[((games['Home.Team'] == team1) & (games['Away.Team'] == team2)
                       ) | ((games['Home.Team'] == team2) & (games['Away.Team'] == team1))].copy()
    
    team_games = team_games.tail(6).iloc[:-1]
    
    team_games['Team1Win'] = ((team_games['Away.Team'] == team1) & (team_games['AwayWin'] == 1)
                             ) | ((team_games['Home.Team'] == team1) & (team_games['HomeWin'] == 1))
    team_games['Team2Win'] = ((team_games['Away.Team'] == team2) & (team_games['AwayWin'] == 1)
                             ) | ((team_games['Home.Team'] == team2) & (team_games['HomeWin'] == 1))
    team1winrate = float(team_games['Team1Win'].astype(int).mean())
    team2winrate = float(team_games['Team2Win'].astype(int).mean())
    return [team1winrate, team2winrate]

In [201]:
 get_last_5_matchups(10970, 'Brisbane', 'Collingwood', games)

[0.4, 0.6]

In [202]:
def add_matchup_columns(games: pd.DataFrame, years) -> pd.DataFrame:
    games_subset = games[games['Season'].isin(years)].copy()

    # Apply get_last_5_matchups to compute win rates for each row
    matchup_winrates = games_subset.apply(
        lambda row: pd.Series(
            get_last_5_matchups(row['Match_id'], row['Home.Team'], row['Away.Team'], games)
        ),
        axis=1
    )

    # Assign column names to the result
    matchup_winrates.columns = ['HomeLast5MatchupWinRate', 'AwayLast5MatchupWinRate']

    # Add the new columns to the original subset
    games_subset[['HomeLast5MatchupWinRate', 'AwayLast5MatchupWinRate']] = matchup_winrates

    # Return the full DataFrame with added columns
    return games_subset

In [203]:
add_matchup_columns(games, [2023])

,Date,Time,Round,Venue,Home.Team,Away.Team,Home.Points,Away.Points,AwayWin,HomeWin,Draw,Season,Match_id,HomeLast5MatchupWinRate,AwayLast5MatchupWinRate
35625,2023-03-16,19:20,1,MCG,Richmond,Carlton,58,58,0,0,1,2023,10751,0.2,0.6
35648,2023-03-17,19:40,1,MCG,Geelong,Collingwood,103,125,1,0,0,2023,10752,0.6,0.4
35671,2023-03-18,13:45,1,Marvel Stadium,North Melbourne,West Coast,87,82,0,1,0,2023,10753,0.6,0.4
35694,2023-03-18,16:05,1,Adelaide Oval,Port Adelaide,Brisbane,126,72,0,1,0,2023,10754,0.2,0.8
35717,2023-03-18,19:00,1,People First Stadium,Gold Coast,Sydney,61,110,1,0,0,2023,10756,0.2,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40475,2023-09-15,19:50,26,MCG,Melbourne,Carlton,71,73,1,0,0,2023,10966,0.4,0.6
40497,2023-09-16,19:10,26,Adelaide Oval,Port Adelaide,GWS,70,93,1,0,0,2023,10967,0.6,0.4
40520,2023-09-22,19:50,27,MCG,Collingwood,GWS,58,57,0,1,0,2023,10968,0.6,0.4
40543,2023-09-23,17:15,27,Gabba,Brisbane,Carlton,79,63,0,1,0,2023,10969,0.8,0.2


In [204]:
def get_last_5_venue(matchid, home_team, away_team, games):
    # Get the venue of the current game
    venue = games.loc[games['Match_id'] == matchid, 'Venue'].values[0]

    # Filter all games involving either team at that venue
    venue_games = games[((games['Home.Team'] == home_team) | (games['Away.Team'] == home_team) |
                         (games['Home.Team'] == away_team) | (games['Away.Team'] == away_team)) &
                        (games['Venue'] == venue)].copy()
    
    venue_games = venue_games.sort_values(['Season', 'Round'])
    # Get last 5 games at venue for home team (excluding current)
    home_venue_games = venue_games[((venue_games['Home.Team'] == home_team) | (venue_games['Away.Team'] == home_team)) &
                                   (venue_games['Match_id'] != matchid)].tail(5)
    # Get last 5 games at venue for away team (excluding current)
    away_venue_games = venue_games[((venue_games['Home.Team'] == away_team) | (venue_games['Away.Team'] == away_team)) &
                                   (venue_games['Match_id'] != matchid)].tail(5)
    # Calculate win rates
    home_wins = ((home_venue_games['Home.Team'] == home_team) & (home_venue_games['HomeWin'] == 1)) | \
                ((home_venue_games['Away.Team'] == home_team) & (home_venue_games['AwayWin'] == 1))
    away_wins = ((away_venue_games['Home.Team'] == away_team) & (away_venue_games['HomeWin'] == 1)) | \
                ((away_venue_games['Away.Team'] == away_team) & (away_venue_games['AwayWin'] == 1))
    home_winrate = home_wins.astype(int).mean() if not home_venue_games.empty else 0.0
    away_winrate = away_wins.astype(int).mean() if not away_venue_games.empty else 0.0
    return [home_winrate, away_winrate]

In [205]:
def add_venue_columns(games: pd.DataFrame, years) -> pd.DataFrame:
    games_subset = games[games['Season'].isin(years)].copy()

    # Apply get_last_5_venue to compute venue win rates for each row
    venue_winrates = games_subset.apply(
        lambda row: pd.Series(
            get_last_5_venue(row['Match_id'], row['Home.Team'], row['Away.Team'], games)
        ),
        axis=1
    )

    # Assign column names
    venue_winrates.columns = ['HomeVenueWinRate', 'AwayVenueWinRate']

    # Add the new columns to the subset
    games_subset[['HomeVenueWinRate', 'AwayVenueWinRate']] = venue_winrates

    return games_subset


Now I'll add a column for days since last game for each team

In [397]:
import pandas as pd

def days_since_last_game(match_id, team, games):
    """
    Calculate days since the last game for a given team and match ID.
    
    Args:
        match_id: The ID of the match to evaluate.
        team: The team name to filter games for.
        games: DataFrame with columns ['Date', 'Home.Team', 'Away.Team', 'Match_id'].
    
    Returns:
        int: Number of days since the team's last game, or 0 if first game.
        None: If team or match_id is not found.
    """
    # Ensure Date is in datetime format
    games = games.copy()
    games['Date'] = pd.to_datetime(games['Date'])
    
    # Filter games where team is either Home or Away, sorted by date
    team_games = games[
        (games['Home.Team'] == team) | (games['Away.Team'] == team)
    ].copy().sort_values('Date').reset_index(drop=True)
    
    # Check if any games were found for the team
    if team_games.empty:
        return None
    
    # Find the current game
    current_game = team_games[team_games['Match_id'] == match_id]
    if current_game.empty:
        return None
    
    # Get the index of the current game
    current_index = current_game.index[0]
    
    # If it's the first game, return 0
    if current_index == 0:
        return 0
    
    # Get the previous game
    previous_game = team_games.iloc[current_index - 1]
    
    # Calculate days between current and previous game
    days_diff = (current_game['Date'].iloc[0] - previous_game['Date']).days
    if days_diff > 30:
        days_diff = 30
    return int(days_diff)

def add_days_columns(games, years):
    """
    Add columns for days since last game for home and away teams in a subset of games.
    
    Args:
        games: DataFrame with game data.
        years: List of seasons to filter (e.g., [2021]).
    
    Returns:
        DataFrame: Subset of games with new columns 'home_days_since' and 'away_days_since'.
    """
    # Filter games for the specified years
    games_subset = games[games['Season'].isin(years)].copy()
    
    # Check if subset is empty
    if games_subset.empty:
        return games_subset
    
    # Calculate days since last game for home and away teams
    games_subset['home_days_since'] = games_subset.apply(
        lambda row: days_since_last_game(row['Match_id'], row['Home.Team'], games),
        axis=1
    )
    games_subset['away_days_since'] = games_subset.apply(
        lambda row: days_since_last_game(row['Match_id'], row['Away.Team'], games),
        axis=1
    )
    games_subset['away_days_since_net'] = games_subset['away_days_since'] - games_subset['home_days_since']
    games_subset['home_days_since_net'] = games_subset['home_days_since'] - games_subset['away_days_since']

    games_subset = games_subset.drop(columns=['home_days_since', 'away_days_since'])

    scaler = MinMaxScaler(feature_range=(0, 1))
    
 
    games_subset[['away_days_since_net', 'home_days_since_net']] = scaler.fit_transform(games_subset[['away_days_since_net', 'home_days_since_net']])



    
    return games_subset


In [398]:
daysdf = add_days_columns(games, [2021])

In [429]:
daysdf['Venue'].unique()

array([' MCG', ' Adelaide Oval', ' Gabba', ' Marvel Stadium',
       ' Optus Stadium', ' ENGIE Stadium', ' GMHBA Stadium', ' SCG',
       ' People First Stadium', ' Manuka Oval', ' Mars Stadium',
       ' UTAS Stadium', ' Ninja Stadium', " Cazaly's Stadium"],
      dtype=object)

Now I will calulate elo ratings

In [422]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

def add_elo_ratings(games: pd.DataFrame, base_elo: int = 1500, k: int = 40) -> pd.DataFrame:
    # Sort by time
    df = games.sort_values(by=['Season', 'Date', 'Time']).copy()

    # Elo dictionary
    elo_ratings = {}
    home_elos, away_elos = [], []

    for _, row in df.iterrows():
        home, away = row['Home.Team'], row['Away.Team']
        margin = abs(row['Home.Points'] - row['Away.Points'])

        # Get ratings
        home_elo = elo_ratings.get(home, base_elo)
        away_elo = elo_ratings.get(away, base_elo)

        # Expected outcome
        expected_home = 1 / (1 + 10 ** ((away_elo - home_elo) / 400))
        expected_away = 1 - expected_home

        # Actual result
        home_score = 1 if row['HomeWin'] == 1 else 0
        away_score = 1 if row['AwayWin'] == 1 else 0

        # Margin of Victory Multiplier
        mov_mult = np.log(margin + 1) * (2.2 / ((home_elo - away_elo) * 0.001 + 2.2))

        # Update Elo ratings
        elo_ratings[home] = home_elo + k * mov_mult * (home_score - expected_home)
        elo_ratings[away] = away_elo + k * mov_mult * (away_score - expected_away)

        # Save pre-game Elo
        home_elos.append(home_elo)
        away_elos.append(away_elo)

    # Add Elo values to df
    df['HomeElo'] = home_elos
    df['AwayElo'] = away_elos
    df['NetHomeElo'] = df['HomeElo'] - df['AwayElo']
    df['NetAwayElo'] = -df['NetHomeElo']

    # Normalize difference features only (not absolute ratings)
    scaler = MinMaxScaler(feature_range=(0, 1))
    df[['NetHomeElo', 'NetAwayElo']] = scaler.fit_transform(df[['NetHomeElo', 'NetAwayElo']])

    return df[['Match_id', 'HomeElo', 'AwayElo', 'NetAwayElo', 'NetHomeElo']]



In [423]:
def add_all_features(games: pd.DataFrame, stats: pd.DataFrame, years: list) -> pd.DataFrame:
    # Compute all features on full history
    ratings_df = add_team_and_net_ratings(games, stats, years)
    forms_df = add_winrate_columns(games, years)
    matchups_df = add_matchup_columns(games, years)
    venues_df = add_venue_columns(games, years)
    days_df = add_days_columns(games, years)
    elo_df = add_elo_ratings(games) 
    # Filter the final set of rows by year
    filtered_games = games[games['Season'].isin(years)].copy()
    # Merge only the needed columns from feature sets
    filtered_games = filtered_games.merge(
        ratings_df[['Match_id', 'HomeNetRating', 'AwayNetRating']],
        on='Match_id', how='left'
    )
    filtered_games = filtered_games.merge(
        forms_df[['Match_id', 'HomeNetForm', 'AwayNetForm']],
        on='Match_id', how='left'
    )
    filtered_games = filtered_games.merge(
        matchups_df[['Match_id', 'HomeLast5MatchupWinRate', 'AwayLast5MatchupWinRate']],
        on='Match_id', how='left'
    )
    filtered_games = filtered_games.merge(
        venues_df[['Match_id', 'HomeVenueWinRate', 'AwayVenueWinRate']],
        on='Match_id', how='left'
    )
    filtered_games = filtered_games.merge(
        days_df[['Match_id', 'home_days_since_net', 'away_days_since_net']],
        on='Match_id', how='left'
    )
    filtered_games = filtered_games.merge(
        elo_df[['Match_id', 'HomeElo', 'AwayElo', 'NetHomeElo', 'NetAwayElo']],
        on='Match_id', how='left'
    )
    
    # Select final columns
    final = filtered_games[[
        'Match_id', 'Season', 'Home.Team', 'Away.Team',
        'Home.Points', 'Away.Points',
        'HomeNetRating', 'AwayNetRating',
        'HomeNetForm', 'AwayNetForm',
        'HomeLast5MatchupWinRate', 'AwayLast5MatchupWinRate',
        'HomeVenueWinRate', 'AwayVenueWinRate',
        'home_days_since_net', 'away_days_since_net', 
        'NetHomeElo', 'NetAwayElo',
        'HomeWin', 'AwayWin'
    ]]
    return final

In [424]:
add_elo_ratings(games)

,Match_id,HomeElo,AwayElo,NetAwayElo,NetHomeElo
0,5964,1500.000000,1500.000000,0.505138,0.494862
1,5965,1500.000000,1500.000000,0.505138,0.494862
2,5966,1500.000000,1500.000000,0.505138,0.494862
3,5967,1500.000000,1500.000000,0.505138,0.494862
4,5968,1500.000000,1500.000000,0.505138,0.494862
...,...,...,...,...,...
2205,11349,1694.387382,1567.755217,0.438670,0.561330
2206,11355,1613.116693,1854.654020,0.631920,0.368080
2207,11357,1688.282565,997.994623,0.142810,0.857190
2208,11350,1170.574864,1640.966539,0.752045,0.247955


In [425]:
final_dataset = add_all_features(games, stats, [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025])

In [426]:
final_dataset = final_dataset[10:]

In [427]:
final_dataset.to_csv('footywire_data.csv', index=False)

In [428]:
final_dataset

,Match_id,Season,Home.Team,Away.Team,Home.Points,Away.Points,HomeNetRating,AwayNetRating,HomeNetForm,AwayNetForm,HomeLast5MatchupWinRate,AwayLast5MatchupWinRate,HomeVenueWinRate,AwayVenueWinRate,home_days_since_net,away_days_since_net,NetHomeElo,NetAwayElo,HomeWin,AwayWin
10,5974,2015,Richmond,Western Bulldogs,66,85,0.480445,0.519555,0.000000,0.000000,0.4,0.6,0.2,0.4,0.553191,0.446809,0.504670,0.495330,0,1
11,5975,2015,GWS,Melbourne,101,56,0.308924,0.691076,0.000000,0.000000,0.6,0.4,0.6,0.5,0.489362,0.510638,0.484434,0.515566,1,0
12,5976,2015,Collingwood,Adelaide,63,90,0.000000,1.000000,0.000000,0.000000,1.0,0.0,0.8,0.6,0.531915,0.468085,0.476052,0.523948,0,1
13,5978,2015,Port Adelaide,Sydney,44,92,0.627328,0.372672,-1.000000,1.000000,0.8,0.2,0.6,0.6,0.489362,0.510638,0.446105,0.553895,0,1
14,5977,2015,Gold Coast,St Kilda,76,104,0.475165,0.524835,0.000000,0.000000,0.2,0.8,0.8,0.6,0.531915,0.468085,0.484434,0.515566,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2205,11349,2025,GWS,Sydney,102,58,0.536344,0.463656,0.111111,-0.111111,0.0,1.0,0.8,0.8,0.553191,0.446809,0.561330,0.438670,1,0
2206,11355,2025,Gold Coast,Brisbane,130,64,0.378855,0.621145,-0.111111,0.111111,0.2,0.8,0.8,0.4,0.468085,0.531915,0.368080,0.631920,1,0
2207,11357,2025,Fremantle,West Coast,126,77,0.544177,0.455823,0.777778,-0.777778,0.8,0.2,1.0,0.2,0.489362,0.510638,0.857190,0.142810,1,0
2208,11350,2025,North Melbourne,Geelong,49,150,0.215680,0.784320,-0.444444,0.444444,0.0,1.0,0.0,0.4,0.531915,0.468085,0.247955,0.752045,0,1


In [430]:
data = final_dataset[~final_dataset['Season'].isin([2015,2016,2017,2018,2019,2020, 2021])]

In [431]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score

X = data[['AwayNetForm', 'AwayNetRating', 'AwayLast5MatchupWinRate', 'AwayVenueWinRate', 'away_days_since_net', 'NetAwayElo']]
y = data['AwayWin']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


In [437]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Best parameters from tuning
best_params = {
    'max_depth': 10,
    'max_features': 'sqrt',
    'min_samples_leaf': 4,
    'min_samples_split': 10,
    'n_estimators': 300,
    'random_state': 42  # for reproducibility
}



best_model = RandomForestClassifier(**best_params)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

print("Tuned Model Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Tuned Model Accuracy: 0.7407407407407407
              precision    recall  f1-score   support

           0       0.75      0.83      0.79        95
           1       0.72      0.61      0.66        67

    accuracy                           0.74       162
   macro avg       0.74      0.72      0.73       162
weighted avg       0.74      0.74      0.74       162



In [470]:
import pandas as pd

def predict_match(home_team, away_team, venue, season, round_n, date, model, feature_data, stats, X_train_columns):
    """
    Predict outcome of a single AFL match.
    
    Parameters:
        home_team (str): Home team name
        away_team (str): Away team name
        venue (str): Venue name
        season (int): Year of match
        ensemble (sklearn model): Trained VotingClassifier
        feature_data (pd.DataFrame): Full dataset with all computed features
        X_train_columns (list): Columns used to train the model
    """
    # Build match dictionary
    match_dict = {
        "Date": date,
        "Match_id": 999999,  # placeholder, won't be used in training
        "Season": season,
        "Home.Team": home_team,
        "Away.Team": away_team,
        "Venue": venue,
        "Round": round_n,
    }
    
    
    match_df = pd.DataFrame([match_dict])
    new_games = pd.concat([games, match_df], ignore_index=True)
    
    stats_df1 = pd.DataFrame([{"Match_id": 999999,
        "Season": season,
        "Team": home_team,
        "Venue": venue,
        "Round": round_n}])
                              
    stats_df2 = pd.DataFrame([{"Match_id": 999999,  
        "Season": season,
        "Team": away_team,
        "Venue": venue,
        "Round": round_n}])

                              
    new_stats = pd.concat([stats, stats_df1, stats_df2])

    match_features = add_all_features(
        new_games,
        stats=new_stats,  
        years=[season]
    ).query("Match_id == 999999")

    # Select features same as training
    X_new = match_features[X_train_columns]

    # Predict
    predicted_class = model.predict(X_new)[0]
    predicted_proba = model.predict_proba(X_new)[0]

    # Build output DataFrame
    result_df = match_features.copy()
    result_df["PredictedWinner"] = "Home" if predicted_class == 0 else "Away"
    result_df["PredictedProb_HomeWin"] = predicted_proba[0]
    result_df["PredictedProb_AwayWin"] = predicted_proba[1]

    return result_df


In [472]:
result = predict_match(
    home_team="Western Bulldogs",
    away_team="GWS",
    venue="Marvel Stadium",
    season=2025,
    round_n=21,
    date='31-07-2025',
    model=best_model,
    feature_data=games,
    stats=stats,
    X_train_columns=X_train.columns
)

#print(result[[
    #"Home.Team", "Away.Team", "PredictedWinner",
    #"PredictedProb_HomeWin", "PredictedProb_AwayWin"
#]])

result

,Match_id,Season,Home.Team,Away.Team,Home.Points,Away.Points,HomeNetRating,AwayNetRating,HomeNetForm,AwayNetForm,...,AwayVenueWinRate,home_days_since_net,away_days_since_net,NetHomeElo,NetAwayElo,HomeWin,AwayWin,PredictedWinner,PredictedProb_HomeWin,PredictedProb_AwayWin
168,999999,2025,Western Bulldogs,GWS,NaN,NaN,NaN,NaN,-0.333333,0.333333,...,0.0,0.510638,0.489362,0.439512,0.560488,NaN,NaN,Home,0.714729,0.285271


In [441]:
games

,Date,Time,Round,Venue,Home.Team,Away.Team,Home.Points,Away.Points,AwayWin,HomeWin,Draw,Season,Match_id
0,2015-04-02,19:20,1,MCG,Carlton,Richmond,78,105,1,0,0,2015,5964
1,2015-04-04,13:40,1,MCG,Melbourne,Gold Coast,115,89,0,1,0,2015,5965
2,2015-04-04,16:35,1,Accor Stadium,Sydney,Essendon,72,60,0,1,0,2015,5966
3,2015-04-04,18:20,1,Gabba,Brisbane,Collingwood,74,86,1,0,0,2015,5967
4,2015-04-04,19:20,1,Marvel Stadium,Western Bulldogs,West Coast,97,87,0,1,0,2015,5968
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2205,2025-07-25,19:50,20,ENGIE Stadium,GWS,Sydney,102,58,0,1,0,2025,11349
2206,2025-07-26,13:20,20,People First Stadium,Gold Coast,Brisbane,130,64,0,1,0,2025,11355
2207,2025-07-26,14:15,20,Optus Stadium,Fremantle,West Coast,126,77,0,1,0,2025,11357
2208,2025-07-26,19:35,20,Marvel Stadium,North Melbourne,Geelong,49,150,1,0,0,2025,11350
